In [179]:
from copy import deepcopy
import networkx as nx

In [1]:
with open('data/input_20.txt') as fh:
    file_input = fh.read().strip()

In [3]:
test_input = """^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$"""

In [13]:
test_input = """^ENWWW(NEEE|SSE(EE|N))$"""

In [74]:
s = "aaa(NEEE|SSE(EE|N)NNEW)g"
test(s)

('! ', 'aaa(NEEE|SSE(EE|N)NNEW)g')
(3, 19)
('! ', 'NEEE|SSE(EE|N)NNEW')
(8, 5)
('! ', 'EE|N')


In [67]:
s[3:22]

'(NEEE|SSE(EE|N)sdf)'

In [193]:
def find_matching_parenthesis(string):
    n = 0
    i = 0
    while i < len(string):
        if string[i] == '(':
            n += 1
        elif string[i] == ')':
            n -= 1
        if n == 0:
            return i
        i += 1

dirs = {
    'N': (0, -1),
    'W': (-1, 0),
    'S': (0, 1),
    'E': (1, 0),
}

def parse(starting_points, string):
    i = 0
    points = deepcopy(starting_points)
    end_points = []
    edges = []
    while i < len(string):
        if string[i] == '(':
            idx = find_matching_parenthesis(string[i:])
            ps, es = parse(points, string[i+1:i+idx])
            points = []
            for p in ps:
                if p not in points:
                    points.append(p[:])
            edges += es
            # points = list(set(points))
            i += idx
        elif string[i] == '|':
            end_points += deepcopy(points)
            points = deepcopy(starting_points)
        elif string[i] in "NESW":
            for p in points:
                x, y = dirs[string[i]]
                edges += ((tuple(p), (p[0]+x, p[1]+y)),)
                #print(p, x, y, p[0], p[1], (tuple(p), (p[0]+x, p[1]+y)),)
                p[0] += x
                p[1] += y               
        i += 1
    end_points += deepcopy(points)
    
    # print(string, starting_points, end_points, edges)
    return (end_points, edges)

def get_path_len(inp):
    ep, edges = parse([[0,0]], inp)
    G = nx.Graph()
    G.add_edges_from(edges)
    dists = nx.shortest_path(G, source=(0, 0))
    return ({target: len(path)-1 for target, path in dists.iteritems()})

In [194]:
ep, edges = parse([[0,0]], "NNNN")
print(edges)

[((0, 0), (0, -1)), ((0, -1), (0, -2)), ((0, -2), (0, -3)), ((0, -3), (0, -4))]


In [195]:
get_path_len("N(E|W)N")

{(-1, -2): 3, (-1, -1): 2, (0, -1): 1, (0, 0): 0, (1, -2): 3, (1, -1): 2}

Test

In [198]:
lengths = get_path_len("^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$")
print(max(lengths.values()))

23


Part A

In [199]:
%%time
lengths = get_path_len(file_input)
print(max(lengths.values()))

4721
CPU times: user 2.36 s, sys: 211 ms, total: 2.57 s
Wall time: 2.74 s


Part B

In [200]:
n = 0
for l in lengths.values():
    if l >= 1000:
        n += 1
print(n)

8281
